In [18]:
import numpy as np
import matplotlib.pyplot as plt
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.operators.sampling.lhs import LHS
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.factory import get_termination
from pymoo.factory import get_crossover
from pymoo.factory import get_mutation

In [19]:
algorithms_names = ['GA', 'NSGA-II', 'DE']

def genetic_algorithm (problem: object, crossover_probability: float, mutation_probability: float, number_genes: int, print_verbose: bool):
    termination = get_termination("n_gen", number_genes)
    algorithm_ga = GA(
        pop_size=100,
        crossover=get_crossover('real_sbx', prob=crossover_probability),
        mutation=get_mutation('real_pm', prob=mutation_probability)
    )
    res_ga = minimize(
        problem,
        algorithm_ga,
        termination,
        seed=1,
        save_history=True,
        verbose=print_verbose
    )
    return res_ga

def nsga2_algorithm (problem: object, crossover_probability: float, mutation_probability: float, number_genes: int, print_verbose: bool):
    termination = get_termination("n_gen", number_genes)
    algorithm_nsga2 = NSGA2(
        pop_size=100,
        crossover=get_crossover('real_sbx', prob=crossover_probability),
        mutation=get_mutation('real_pm', prob=mutation_probability)
    )
    res_nsga2 = minimize(
        problem,
        algorithm_nsga2,
        termination,
        seed=1,
        save_history=True,
        verbose=print_verbose
    )
    return res_nsga2

def de_algorithm (problem: object, number_genes: int, print_verbose: bool):
    termination = get_termination("n_gen", number_genes)
    algorithm_de = DE(
        pop_size=100,
        sampling=LHS(),
        variant="DE/rand/1/bin",
        CR=0.3,
        dither="vector",
        jitter=False
    ) 
    res_de = minimize(
        problem,
        algorithm_de,
        termination,
        seed=1,
        save_history=True,
        verbose=print_verbose
    )
    return res_de


# Design of Pressure Vessel

Optimization problem: **minimization**

A cylindrical vessel is capped at both ends by hemispherical heads as shown in Fig. 4.1.
The objective is to minimize the total cost, including the cost of the 
material, forming and welding. There are four design variables: thickness of the
shell (Ts ), thickness of the head (Th), inner radius (R) and length of the cylindrical
section of the vessel, not including the head (L) so design vector 
X = (x1 , x2 , x3 , x4 ) = (Ts , Th , R, L). Ts and Th are integer multiples of 0.0625 inch,
which are the available thicknesses of rolled steel plates, and R and L are continuous.
( [Mechanical Design optimization Using Advanced Optimization Techniques, 4.3.1 Example 8: Design of Pressure Vessel](https://libgen.is/book/index.php?md5=611FEA61F6BCB661048FC05D01C78C5E) ) 

![alt text](doc/image/presure-vessel/image.png "Title")

## Description

- Dimension of optimization problem: **4 Dimension**
- Population size: **100 members**
- Stopping criterion: **30 generations**
- Number of runs: **30**

## Equations

![alt text](doc/image/presure-vessel/eq-1.png "Title")

![alt text](doc/image/presure-vessel/eq-2.png "Title")




In [20]:
class PressureVessel (ElementwiseProblem):
    
    def __init__(self):
        super().__init__(n_var=4,
                         n_obj=1,
                         n_constr=4,
                         xl=np.array([0.0625, 0.0625, 10, 10]),
                         xu=np.array([6.1875, 6.1875, 200, 200]))

    def _evaluate(self, x, out, *args, **kwargs):
        x[0] = np.round(x[0] / 0.0625, 0) * 0.0625
        x[1] = np.round(x[1] / 0.0625, 0) * 0.0625
        
        # function
        f = 0.6224 * x[0] * x[2] * x[3] + \
            1.7781 * x[1] * (x[2] ** 2) + \
            3.1661 * (x[0] ** 2) * x[3] + \
            19.84*(x[0] ** 2) * x[2]

        # constraints 
        g1 = -x[0] + 0.0193 * x[2]
        g2 = -x[1] + 0.00954 * x[2]
        g3 = -np.round(np.pi, 5) * x[2] ** 2 * x[3] - \
             (4/3) * np.round(np.pi, 5) * x[2] ** 3 + \
             1296000
        g4 = x[3] - 240

        out["F"] = [f]
        out["G"] = [g1, g2, g3, g4]

In [21]:
problem = PressureVessel()

res_algorithm_ga = genetic_algorithm(problem, 0.9, 0.01, 30, False)
res_algorithm_nsga2 = nsga2_algorithm(problem, 0.9, 0.01, 30, False)
res_algorithm_de = de_algorithm(problem, 30, False)
print('------------------------------------------------------')
print('f(x) = ', res_algorithm_ga.F[0], algorithms_names[0])
print('f(x) = ', res_algorithm_nsga2.F[0], algorithms_names[1])
print('f(x) = ', res_algorithm_de.F[0], algorithms_names[2], '\n')

print('X =', res_algorithm_ga.X, algorithms_names[0])
print('X =', res_algorithm_nsga2.X, algorithms_names[1])
print('X =', res_algorithm_de.X, algorithms_names[2])
print('------------------------------------------------------\n')

------------------------------------------------------
f(x) =  7005.639333138592 GA
f(x) =  7364.541198366935 NSGA-II
f(x) =  7130.404049226436 DE 

X = [  1.0060607    0.5307703   46.29381148 130.76655942] GA
X = [ 1.28118274  0.59905755 64.56064058 13.25098788] NSGA-II
X = [  1.0189566    0.51914981  47.44110618 128.10594235] DE
------------------------------------------------------



# Design of Tension/Compression Spring

Optimization problem: **minimization**

This problem is taken from Belegundu which consists of minimizing the weight of
a tension/compression spring as shown in Fig. 4.3) subject to constraints on minimum
deflection, shear stress, surge frequency, limits on outside diameter and on design
variables. The design variables are the wire diameter (d), the mean coil diameter
(D) and the number of active coils (N). Design vector can be defined as X = (x1 , x2 , x3 ) = (d, D, N).
( [Mechanical Design optimization Using Advanced Optimization Techniques, 4.3.3 Example 10: Design of Tension/Compression Spring](https://libgen.is/book/index.php?md5=611FEA61F6BCB661048FC05D01C78C5E) ) 

![alt text](doc/image/tension-compression-spring/image.png "Title")

## Description

- Dimension of optimization problem: **3 Dimension**
- Population size: **100 members**
- Stopping criterion: **30 generations**
- Number of runs: **30**

## Equations

![alt text](doc/image/tension-compression-spring/eq-1.png "Title")

![alt text](doc/image/tension-compression-spring/eq-2.png "Title")



In [22]:
class TensionCompressionSpring (ElementwiseProblem):
    
    def __init__(self):
        super().__init__(n_var=3,
                         n_obj=1,
                         n_constr=4,
                         xl=np.array([0.05, 0.25, 2]),
                         xu=np.array([2, 1.3, 15]))

    def _evaluate(self, x, out, *args, **kwargs):

        # function
        f = (x[2] + 2) * x[1] * x[0] ** 2

        # constraints 
        g1 = 1 - ((x[1] ** 3 * x[2]) / (71785 * x[0] ** 4))
        g2 = ((4 * x[1] ** 2 - x[0] * x[1]) / (12566 * (x[1] * x[0] ** 3 - x[0] ** 4))) + (1 / (5108 * x[0] ** 2)) - 1
        g3 = 1 - ((140.45 * x[0]) / (x[1] ** 2 * x[2]))
        g4 = ((x[1] + x[0]) / 1.5) - 1

        out["F"] = [f]
        out["G"] = [g1, g2, g3, g4]

In [40]:
problem = TensionCompressionSpring()

res_algorithm_ga = genetic_algorithm(problem, 0.9, 0.01, 30, False)
res_algorithm_nsga2 = nsga2_algorithm(problem, 0.9, 0.01, 30, False)
res_algorithm_de = de_algorithm(problem, 30, False)

print(len(res_algorithm_ga.history))
print(res_algorithm_ga.history[0].pop.get('F').std())
print('------------------------------------------------------')
print('f(x) = ', res_algorithm_ga.F[0], algorithms_names[0])
print('f(x) = ', res_algorithm_nsga2.F[0], algorithms_names[1])
print('f(x) = ', res_algorithm_de.F[0], algorithms_names[2], '\n')

print('X =', res_algorithm_ga.X, algorithms_names[0])
print('X =', res_algorithm_nsga2.X, algorithms_names[1])
print('X =', res_algorithm_de.X, algorithms_names[2])
print('------------------------------------------------------\n')

30
12.415047596723348
------------------------------------------------------
f(x) =  0.021372281192056773 GA
f(x) =  0.019302249194294203 NSGA-II
f(x) =  0.013070176170847716 DE 

X = [0.06757569 0.84379717 3.54666405] GA
X = [0.06676148 0.81976985 3.28279942] NSGA-II
X = [ 0.05248281  0.37283837 10.72701974] DE
------------------------------------------------------



# Design of a Speed Reducer

Optimization problem: **minimization**

The weight of the speed reducer as shown in Fig. 4.4 is to be minimized subject to
constraints on bending stress of the gear teeth, surfaces stress, transverse deflec-
tions of the shafts and stresses in the shafts. The variables x 1 , x 2 , x 3 , x 4 , x 5 , x 6 and
x 7 are the face width, module of teeth, number of teeth in the pinion, length of the
first shaft between bearings, length of the second shaft between bearings and the
diameter of the first and second shafts, respectively. The third variable is integer,
the rest of them are continuous.
( [Mechanical Design optimization Using Advanced Optimization Techniques, 4.3.4 Example 11: Design of a Speed Reducer](https://libgen.is/book/index.php?md5=611FEA61F6BCB661048FC05D01C78C5E) ) 

![alt text](doc/image/speed-reducer/image.png "Title")

## Description

- Dimension of optimization problem: **7 Dimension**
- Population size: **100 members**
- Stopping criterion: **30 generations**
- Number of runs: **30**

## Equations

![alt text](doc/image/speed-reducer/eq-1.png "Title")

![alt text](doc/image/speed-reducer/eq-2.png "Title")

In [24]:
class SpeedReducer(ElementwiseProblem):
    
    def __init__(self):
        super().__init__(n_var=7,
                         n_obj=1,
                         n_constr=11,
                         xl=np.array([2.6, 0.7, 17, 7.3, 7.8, 2.9, 5.0]),
                         xu=np.array([3.6, 0.8, 28, 8.3, 8.3, 3.9, 5.5]))

    def _evaluate(self, x, out, *args, **kwargs):
        x1, x2, x3, x4, x5, x6, x7 = tuple(x)

        # function
        f = 0.7854 * x1 * (x2 ** 2) * \
            ( 3.3333 * (x3 ** 2) + 14.9334 * x3 - 43.0934) - \
            1.508 * x1 * ( x6 ** 2 + x7 ** 2 ) + \
            7.4777 * ( x6 ** 3 + x7 ** 3 ) + \
            0.7854 * (x4 * (x6 ** 2) + x5 * (x7**2) )

        # constraints 
        g1  = 27/(x1 * ( x2 ** 2 ) * x3) - 1
        g2  = 397.5/(x1 * ( x2 ** 2 ) * (x3 ** 2)) - 1
        g3  = (1.93 * ( x4 ** 3 ))/(x2 * x3  * (x6 ** 4)) - 1
        g4  = (1.93 * ( x5 ** 3 ))/(x2 * x3  * (x7 ** 4)) - 1
        g5  = np.sqrt( ( (745*x4)/(x2*x3) )**2 + 16.9e6 )/(110 * (x6 ** 3)) - 1
        g6  = np.sqrt( ( (745*x5)/(x2*x3) )**2 + 157.5e6 )/(85 * (x7 ** 3)) - 1
        g7  = x2*x3/40 - 1
        g8  = (5*x2)/x1 - 1
        g9  = x1/(12*x2) - 1
        g10 = (1.5*x6 + 1.9)/x4 - 1
        g11 = (1.1*x7 + 1.9)/x5 - 1

        out["F"] = [f]
        out["G"] = [g1, g2, g3, g4, g5, g6, g7, g8, g9, g10, g11]

In [25]:
problem = SpeedReducer()

res_algorithm_ga = genetic_algorithm(problem, 0.9, 0.01, 30, False)
res_algorithm_nsga2 = nsga2_algorithm(problem, 0.9, 0.01, 30, False)
res_algorithm_de = de_algorithm(problem, 30, False)
print('------------------------------------------------------')
print('f(x) = ', res_algorithm_ga.F[0], algorithms_names[0])
print('f(x) = ', res_algorithm_nsga2.F[0], algorithms_names[1])
print('f(x) = ', res_algorithm_de.F[0], algorithms_names[2], '\n')

print('X =', res_algorithm_ga.X, algorithms_names[0])
print('X =', res_algorithm_nsga2.X, algorithms_names[1])
print('X =', res_algorithm_de.X, algorithms_names[2])
print('------------------------------------------------------\n')

------------------------------------------------------
f(x) =  3020.3830194738443 GA
f(x) =  3013.8811753342884 NSGA-II
f(x) =  3050.1111001476274 DE 

X = [ 3.53413725  0.70005293 17.02903985  7.7022275   7.86331574  3.35137794
  5.28680934] GA
X = [ 3.51393004  0.70023627 17.00974799  7.70561718  7.90544358  3.36289944
  5.28689827] NSGA-II
X = [ 3.50619749  0.70120773 17.05981058  8.29165599  8.01886098  3.40845253
  5.29686587] DE
------------------------------------------------------

